## 타이타닉 생존자 예측

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from matplotlib import font_manager, rc # Only for mac
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import titanic

In [2]:
# MAC 한글 깨짐 처리
## Windows 환경에서는 반드시 제외!!
rc('font', family = 'AppleGothic')

In [3]:
df = pd.read_csv('train_titanic.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df2 = df

## NULL 값 처리
df['Age'].fillna(df2['Age'].mean(), inplace=True)
df2['Cabin'].fillna('N', inplace = True)

## 잔여 NULL 유무 확인
df2.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

In [5]:
## SEX 분포
df2['Sex'].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [6]:
## Cabin 분포
print(df2['Cabin'].value_counts())

Cabin
N              687
C23 C25 C27      4
G6               4
B96 B98          4
C22 C26          3
              ... 
E34              1
C7               1
C54              1
E36              1
C148             1
Name: count, Length: 148, dtype: int64


In [7]:
## Embarked 분포
df2['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [8]:
# Cabin 분포 재가공
## N이 가장 많고, 속성값이 제대로 정리되지 않았기 때문

df2['Cabin'] = df['Cabin'].str[:1]
df2['Cabin'].value_counts()

Cabin
N    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: count, dtype: int64

In [9]:
# 성별에 따른 생존 여부
g = df2.groupby(['Sex', 'Survived'])
g['Survived'].count()

Sex     Survived
female  0            81
        1           233
male    0           468
        1           109
Name: Survived, dtype: int64

In [11]:
# 객실등급에 따른 생존 여부
p = df2.groupby(['Pclass', 'Survived'])
p['Survived'].count()

Pclass  Survived
1       0            80
        1           136
2       0            97
        1            87
3       0           372
        1           119
Name: Survived, dtype: int64

In [12]:
# 연령별 생존 여부
a = df2.groupby(['Age', 'Survived'])
a['Survived'].count()

Age    Survived
0.42   1           1
0.67   1           1
0.75   1           2
0.83   1           2
0.92   1           1
                  ..
70.00  0           2
70.50  0           1
71.00  0           2
74.00  0           1
80.00  1           1
Name: Survived, Length: 144, dtype: int64

In [ ]:
plt.figure()
plt.title("남녀 생존 비율")
ax = plt.gca()
sns.barplot(x='Sex', y='Survived', data=df, ax = ax)
plt.show()

In [ ]:
plt.figure()
plt.title("객실 등급에 따른 생존 비율")
ax = plt.gca()
sns.barplot(x='Pclass', y = 'Survived', hue = 'Sex', data = df, ax = ax)
plt.show()

In [ ]:
df['Age_cate'] = df2['Age'].apply(lambda x:titanic.get_category(x))

In [ ]:
plt.figure()

ax = plt.gca()
sns.barplot(x="Age_cate", y="Survived", hue = "Sex", data = df, ax = ax)
plt.ylim(0,1.4)
plt.show()

In [ ]:
df2 = titanic.encode_features(df2, ['Cabin', 'Sex', 'Embarked', 'Age_cate'])
df2[['Cabin', 'Sex', 'Embarked', 'Age_cate']].head(5)

In [ ]:
df2['CN'] = df2['Name'].apply(lambda x:titanic.get_name_index(x))
df2['CN'].value_counts()

In [ ]:
df3 = df2.drop(['PassengerId', 'Name', 'Ticket', 'Age'], axis=1)
df3.info()

In [ ]:
xdf = df3.drop('Survived', axis=1)
ydf = df3['Survived']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(xdf,ydf)

In [ ]:
## 로지스틱 회귀 분류기
lr = LogisticRegression(solver='liblinear')
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
accuracy_score(y_test, lr_pred)

In [ ]:
df3.corr()

In [ ]:
lr = LogisticRegression(solver='liblinear')
cross_val_score(lr, xdf, ydf, cv = 5)

In [ ]:
lr = LogisticRegression(solver='liblinear')
titanic.exec_kfold(lr, xdf, ydf)

### 파라미터 대입
#### 최적화된 파라미터를 대입하여 DecisionTreeClassifier를 학습시킨 뒤 예측 정확도 확인

In [ ]:
parameters = {'max_depth':[2,3,5,10],
              'min_samples_split':[2,3,5],
              'min_samples_leaf':[1,5,8]}

dtc = DecisionTreeClassifier()
grid_model = GridSearchCV(dtc, param_grid=parameters, scoring='accuracy', cv=5)
grid_model.fit(x_train, y_train)

print("GridSearchCV 최고 하이퍼 파라미터: ", grid_model.best_params_)
print("GridSearchCV 최고 정확도: ", grid_model.best_score_)

best_model = grid_model.best_estimator_
pred_val = best_model.predict(x_test)
print("TestData DecisionTreeClassifier 정확도: ", accuracy_score(y_test, pred_val))

#### Test data에 DecisionTreeClassifier 학습 이후 예측 정확도 상승 확인